<a href="https://colab.research.google.com/github/DevNicque/MyCods/blob/master/C%C3%B3pia_de_base_atem_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configurações

In [3]:
!pip install elasticsearch

     |████████████████████████████████| 327kB 4.2MB/s 


In [4]:
import pandas as pd
from elasticsearch import Elasticsearch
import json

In [5]:
def send_elasticsearch(index_name, id, documents):
        elasticsearch_username = 'elastic'
        elasticsearch_password = 'gxjzXnMQ9ZGHE9Huc7'
        elasticsearch_host = '159.89.82.132'
        es = Elasticsearch(server_hostname=[elasticsearch_host],
                        http_auth=(elasticsearch_username,
                        elasticsearch_password),scheme="https",
                        hosts=[elasticsearch_host],port=9300,
                        verify_certs=False)
        es.indices.create(index=index_name, ignore=400)
        es.index(index=index_name, id=id, body=documents, doc_type='_doc')

#Importação dos dados

In [6]:
df = pd.read_csv('atem.csv')
df.head()

,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1,Digite os dados do pesquisado,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Qual o cargo do respondente?,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Qual o segmento?,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,"Quanto a prestação de serviço do seu assessor comercial, classifique os seguintes itens:",Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,"Em linhas gerais, como considera o relacionamento com o seu assessor comercial:",Unnamed: 38,"SE ENTREGA responder as perguntas 6 e 7Em caso de entrega do produto no posto ou empresa do cliente, classifique o serviço prestado pela ATEM nos itens abaixo:",Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Em linhas gerais como considera o serviço de entrega da ATEM?,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,SE RETIRA responder as perguntas 8 e 9Em caso de retirada do produto na base da ATEM:,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Em linhas gerais como considera o serviço de retirada do produto na base da ATEM?,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,"Quanto a Central de Atendimento da ATEM, classifique os seguintes itens:",Unnamed: 62,Unnamed: 63,"Você já fez alguma reclamação ou solicitação na CA?Se SIM, responder P12 e P13. Se NÃO pular para P14","Em caso positivo para a pergunta 10, quantas vezes já abriu chamado na central:",Quanto a sua reclamação ou solicitação.,Unnamed: 67,Em linhas gerais como considera o serviço da Central de Atendimento da ATEM:,Unnamed: 69,"(Somente posto ATEM e pontão ATEM) Quanto as ações promocionais de Marketing, você observa que:",Unnamed: 71,Unnamed: 72,Observações da Pesquisa: (Atendente)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nome,Empresa,Endereço,Endereço 2,Cidade/Município,Estado,Código postal,País,Endereço de email,Número de telefone,Diretor (a),Gerente,Coordenador (a),Assistente,Outro (especifique),ATEM,COF (Consumidor Final),COFG (Consumidor Final Governamental),POSBB (Postos Bandeira Branca),PTAT (Pontão ATEM),PTBB (pontão Bandeira Branca),TRR (Transportador Revendedor Retalista),TRRNI (Transportador Revendedor Retalista na N...,Cordialidade ( é gentil e atencioso nos atendi...,Disponibilidade ( atende fora do dia de visita...,Eficácia ( lhe entrega todas as informações e ...,Eficiência ( traz soluções adequadas para o se...,Orientações Técnicas Legais ( quando necessári...,Response,Por que? comente,Cordialidade do Motorista (ele foi gentil e at...,Entrega no Prazo (o produto foi entregue dentr...,O Ciclo do Pedido é adequado? (tempo decorrido...,Condições de Frota (o estado de manutenção pas...,Procedimentos de segurança no momento da entre...,5.0,4.0,3.0,2.0,1.0,Por que? Comente,Cordialidade do atendimento (o atendimento na ...,Mercadoria disponível no Prazo (o produto esta...,Tempo do ciclo do pedido é adequado? (entre a ...,Procedimentos de segurança no momento da entre...,Há uma comunicação de agenda eficiente?,5.0,4.0,3.0,2.0,1.0,Por que? comente.,Cordialidade ( o atendimento da atendente foi ...,Eficácia ( tempo de espera e tempo de resposta...,Eficiência ( o suporte atendeu suas solicitaçõ...,Response,Response,Recebeu retorno em até 48h?,Foi apresentada solução da sua solicitação?,Response,Por que atribuiu esta nota? Comente,há interesse dos clientes nas ações?,há aumento de vendas no período?,há conquista de novos clientes?,Open-Ended Response
1,1.226668e+10,397988367.0,12/17/2020 01:46:57 PM,12/17/2020 03:05:15 PM,177.149.116.220,NaN,NaN,NaN,NaN,ITAMIR ARDUZO,CMJ COMERCIO DE COMBUSTIVEIS LTDA,NaN,NaN,LUCAS DO RIO,MT,NaN,NaN,NaN,6599936-9977,NaN,Gerente,NaN,NaN,NaN,NaN,NaN,NaN,POSBB (Postos Bandeira Branca),NaN,NaN,NaN,NaN,5,5,5,5,5,5,CLIENTE É BEM ATENDIDO NÃO TEVE PROBLEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,4,5,5,5,NaN,4.0,NaN,NaN,NaN,AS VEZES ATRASA FORA ISSO OTIMO A

In [7]:
df = df.fillna(value=1000)
df.head()

,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1,Digite os dados do pesquisado,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Qual o cargo do respondente?,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Qual o segmento?,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,"Quanto a prestação de serviço do seu assessor comercial, classifique os seguintes itens:",Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,"Em linhas gerais, como considera o relacionamento com o seu assessor comercial:",Unnamed: 38,"SE ENTREGA responder as perguntas 6 e 7Em caso de entrega do produto no posto ou empresa do cliente, classifique o serviço prestado pela ATEM nos itens abaixo:",Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Em linhas gerais como considera o serviço de entrega da ATEM?,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,SE RETIRA responder as perguntas 8 e 9Em caso de retirada do produto na base da ATEM:,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Em linhas gerais como considera o serviço de retirada do produto na base da ATEM?,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,"Quanto a Central de Atendimento da ATEM, classifique os seguintes itens:",Unnamed: 62,Unnamed: 63,"Você já fez alguma reclamação ou solicitação na CA?Se SIM, responder P12 e P13. Se NÃO pular para P14","Em caso positivo para a pergunta 10, quantas vezes já abriu chamado na central:",Quanto a sua reclamação ou solicitação.,Unnamed: 67,Em linhas gerais como considera o serviço da Central de Atendimento da ATEM:,Unnamed: 69,"(Somente posto ATEM e pontão ATEM) Quanto as ações promocionais de Marketing, você observa que:",Unnamed: 71,Unnamed: 72,Observações da Pesquisa: (Atendente)
0,1.000000e+03,1000.0,1000,1000,1000,1000.0,1000.0,1000.0,1000.0,Nome,Empresa,Endereço,Endereço 2,Cidade/Município,Estado,Código postal,País,Endereço de email,Número de telefone,Diretor (a),Gerente,Coordenador (a),Assistente,Outro (especifique),ATEM,COF (Consumidor Final),COFG (Consumidor Final Governamental),POSBB (Postos Bandeira Branca),PTAT (Pontão ATEM),PTBB (pontão Bandeira Branca),TRR (Transportador Revendedor Retalista),TRRNI (Transportador Revendedor Retalista na N...,Cordialidade ( é gentil e atencioso nos atendi...,Disponibilidade ( atende fora do dia de visita...,Eficácia ( lhe entrega todas as informações e ...,Eficiência ( traz soluções adequadas para o se...,Orientações Técnicas Legais ( quando necessári...,Response,Por que? comente,Cordialidade do Motorista (ele foi gentil e at...,Entrega no Prazo (o produto foi entregue dentr...,O Ciclo do Pedido é adequado? (tempo decorrido...,Condições de Frota (o estado de manutenção pas...,Procedimentos de segurança no momento da entre...,5.0,4.0,3.0,2.0,1.0,Por que? Comente,Cordialidade do atendimento (o atendimento na ...,Mercadoria disponível no Prazo (o produto esta...,Tempo do ciclo do pedido é adequado? (entre a ...,Procedimentos de segurança no momento da entre...,Há uma comunicação de agenda eficiente?,5.0,4.0,3.0,2.0,1.0,Por que? comente.,Cordialidade ( o atendimento da atendente foi ...,Eficácia ( tempo de espera e tempo de resposta...,Eficiência ( o suporte atendeu suas solicitaçõ...,Response,Response,Recebeu retorno em até 48h?,Foi apresentada solução da sua solicitação?,Response,Por que atribuiu esta nota? Comente,há interesse dos clientes nas ações?,há aumento de vendas no período?,há conquista de novos clientes?,Open-Ended Response
1,1.226668e+10,397988367.0,12/17/2020 01:46:57 PM,12/17/2020 03:05:15 PM,177.149.116.220,1000.0,1000.0,1000.0,1000.0,ITAMIR ARDUZO,CMJ COMERCIO DE COMBUSTIVEIS LTDA,1000,1000,LUCAS DO RIO,MT,1000,1000,1000,6599936-9977,1000,Gerente,1000,1000,1000,1000,1000,1000,POSBB (Postos Bandeira Branca),1000,1000,1000,1000,5,5,5,5,5,5,CLIENTE É BEM ATENDIDO NÃO TEVE PROBLEMA,1000,1000,1000,1000,1000,1000.0,1000.0,1000.0,1000.0

#Tratamento dos Dados

**Tratamento da tabela geral**

In [27]:
df_geral = df.loc[:,'date_created']
df_geral = df_geral.drop(0)
df_geral.index = df_geral.index.rename('ID')


df_geral.describe

<bound method NDFrame.describe of ID
1      12/17/2020 01:46:57 PM
2      12/17/2020 02:58:55 PM
3      12/17/2020 02:22:27 PM
4      12/17/2020 02:33:53 PM
5      12/17/2020 01:52:50 PM
                ...          
178    12/14/2020 03:46:41 PM
179    12/14/2020 03:45:21 PM
180    12/14/2020 03:43:22 PM
181    12/14/2020 03:41:24 PM
182    12/14/2020 03:41:25 PM
Name: date_created, Length: 182, dtype: object>

**Tratamento da tabela cargo**

In [28]:
df3 = df.loc[:,'Qual o cargo do respondente?':'Unnamed: 23']
df3.columns = df3.loc[0]
df3 = df3.drop(0)
df3.columns = ['diretor', 'gerente', 'coordenador', 'assistente', 'outro']
df3.index = df3.index.rename('ID')
#df3 = df3.fillna(value=1)


df3.head()

,diretor,gerente,coordenador,assistente,outro
ID,,,,,
1,1000,Gerente,1000,1000,1000
2,1000,1000,1000,1000,1000
3,1000,1000,1000,1000,1000
4,1000,1000,1000,1000,1000
5,Diretor (a),1000,1000,1000,1000


**Tratamento da tabela segmento**

In [29]:
df4 = df.loc[:,'Qual o segmento?':'Unnamed: 31']
df4.columns = df4.loc[0]
df4 = df4.drop(0)

df4.columns = ['ATEM', 'COF', 'COFG', 'POSBB', 'PTAT','PTBB','TRR', 'TRRNI']
df4.index = df4.index.rename('ID')
df4.head()

,ATEM,COF,COFG,POSBB,PTAT,PTBB,TRR,TRRNI
ID,,,,,,,,
1,1000,1000,1000,POSBB (Postos Bandeira Branca),1000,1000,1000,1000
2,1000,1000,1000,1000,1000,1000,1000,1000
3,1000,1000,1000,1000,1000,1000,1000,1000
4,1000,1000,1000,1000,1000,1000,1000,1000
5,1000,1000,1000,POSBB (Postos Bandeira Branca),1000,1000,1000,1000


**Tratamento da tabela Dados dos Pesquisados**

In [31]:
df2 = df.loc[:,'Digite os dados do pesquisado':'Unnamed: 18']
df2.columns = df2.loc[0]
df2 = df2.drop(0)
df2.columns = ['nome', 'empresa','endereco','endereco2','cidade','estado','codigo_postal', 'pais','email','telefone']
df2.index = df2.index.rename('ID')

df2.head()

,nome,empresa,endereco,endereco2,cidade,estado,codigo_postal,pais,email,telefone
ID,,,,,,,,,,
1,ITAMIR ARDUZO,CMJ COMERCIO DE COMBUSTIVEIS LTDA,1000,1000,LUCAS DO RIO,MT,1000,1000,1000,6599936-9977
2,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825
3,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825
4,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825
5,ORGIER,O L QUEIROZ,1000,1000,CAROEBE,RORAIMA,1000,1000,1000,(95) 999711717


**Incluindo campo cargo**

In [32]:
aux = 1000
df2['Cargo'] =['Diretor' if a != aux
              else 'Gerente' if b != aux
              else 'Coordenador' if c != aux
              else 'Assistente' if d != aux
              else 'Outro' if e != aux
              else 'Não Informado'
              for a,b,c,d,e in zip(df3['diretor'].to_list(), df3['gerente'].to_list(), df3['coordenador'].to_list(), df3['assistente'].to_list(), df3['outro'].to_list())]

df2.head()

,nome,empresa,endereco,endereco2,cidade,estado,codigo_postal,pais,email,telefone,Cargo
ID,,,,,,,,,,,
1,ITAMIR ARDUZO,CMJ COMERCIO DE COMBUSTIVEIS LTDA,1000,1000,LUCAS DO RIO,MT,1000,1000,1000,6599936-9977,Gerente
2,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825,Não Informado
3,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825,Não Informado
4,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825,Não Informado
5,ORGIER,O L QUEIROZ,1000,1000,CAROEBE,RORAIMA,1000,1000,1000,(95) 999711717,Diretor


**Incluindo campo segmento**

In [33]:
df2['Segmento'] =['ATEM' if a != aux
              else 'COF' if b != aux
              else 'COFG' if c != aux
              else 'POSBB' if d != aux
              else 'PTAT' if e != aux
              else 'PTBB' if e != aux
              else 'TRR' if e != aux
              else 'TRRNI' if e != aux
              else 'Não Informado'
              for a,b,c,d,e,f,g,h in zip(df4['ATEM'].to_list(), df4['COF'].to_list(), df4['COFG'].to_list(), df4['POSBB'].to_list(), df4['PTAT'].to_list(), df4['PTBB'].to_list(), df4['TRR'].to_list(), df4['TRRNI'].to_list())]
               

df2.head()

,nome,empresa,endereco,endereco2,cidade,estado,codigo_postal,pais,email,telefone,Cargo,Segmento
ID,,,,,,,,,,,,
1,ITAMIR ARDUZO,CMJ COMERCIO DE COMBUSTIVEIS LTDA,1000,1000,LUCAS DO RIO,MT,1000,1000,1000,6599936-9977,Gerente,POSBB
2,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825,Não Informado,Não Informado
3,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825,Não Informado,Não Informado
4,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825,Não Informado,Não Informado
5,ORGIER,O L QUEIROZ,1000,1000,CAROEBE,RORAIMA,1000,1000,1000,(95) 999711717,Diretor,POSBB


**Dados do Pesquisado - Tabela Final**

---



In [34]:
df2 = pd.merge(df2,df_geral, on='ID', left_index=True)

df2.to_json(f'dados_do_pesquisado.json',orient="records")

df2.head()

,nome,empresa,endereco,endereco2,cidade,estado,codigo_postal,pais,email,telefone,Cargo,Segmento,date_created
ID,,,,,,,,,,,,,
1,ITAMIR ARDUZO,CMJ COMERCIO DE COMBUSTIVEIS LTDA,1000,1000,LUCAS DO RIO,MT,1000,1000,1000,6599936-9977,Gerente,POSBB,12/17/2020 01:46:57 PM
2,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825,Não Informado,Não Informado,12/17/2020 02:58:55 PM
3,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825,Não Informado,Não Informado,12/17/2020 02:22:27 PM
4,1000,LEONISIA N N BARROSO,1000,1000,CUIABA,MT,1000,1000,1000,(065) 99284-7825,Não Informado,Não Informado,12/17/2020 02:33:53 PM
5,ORGIER,O L QUEIROZ,1000,1000,CAROEBE,RORAIMA,1000,1000,1000,(95) 999711717,Diretor,POSBB,12/17/2020 01:52:50 PM


**Tratamento das Tabelas de Avaliação**

---






**Tabela Avaliação do atendimento do acessor comercial**

In [35]:
df5 = df.loc[:,'Quanto a prestação de serviço do seu assessor comercial, classifique os seguintes itens:':'Unnamed: 36']
df5.columns = df5.loc[0]
df5 = df5.drop(0)
df5.columns = ['cordialidade','disponibilidade','eficacia','eficiencia','orientacoes_tecnicas_legais']
df5.index = df5.index.rename('ID')

df5 = pd.merge(df5,df_geral, on='ID', left_index=True)

df5.to_json(f'avaliacao_atendimento_acessorcomercial.json',orient="records")

df5.head()

,cordialidade,disponibilidade,eficacia,eficiencia,orientacoes_tecnicas_legais,date_created
ID,,,,,,
1,5,5,5,5,5,12/17/2020 01:46:57 PM
2,1000,1000,1000,1000,1000,12/17/2020 02:58:55 PM
3,1000,1000,1000,1000,1000,12/17/2020 02:22:27 PM
4,1000,1000,1000,1000,1000,12/17/2020 02:33:53 PM
5,5,5,5,5,5,12/17/2020 01:52:50 PM


**Tabela de Avaliação em nota relacionamento com o acessor comercial**

In [36]:
df6 = df.loc[:,'Em linhas gerais, como considera o relacionamento com o seu assessor comercial:':'Unnamed: 38']
df6.columns = df6.loc[0]
df6 = df6.drop(0)

df6.columns = ['nota', 'comentario']
df6.index = df6.index.rename('ID')

df6 = pd.merge(df6,df_geral, on='ID', left_index=True)

df6.to_json(f'avaliacao_nota_relacionamento_acessorcomercial.json',orient="records")

df6.head()

,nota,comentario,date_created
ID,,,
1,5,CLIENTE É BEM ATENDIDO NÃO TEVE PROBLEMA,12/17/2020 01:46:57 PM
2,1000,1000,12/17/2020 02:58:55 PM
3,1000,1000,12/17/2020 02:22:27 PM
4,1000,1000,12/17/2020 02:33:53 PM
5,5,MUITO BOM,12/17/2020 01:52:50 PM


**Tabela Avaliação do serviço de entrega**

In [37]:
df7 = df.loc[:,'SE ENTREGA responder as perguntas 6 e 7Em caso de entrega do produto no posto ou empresa do cliente, classifique o serviço prestado pela ATEM nos itens abaixo:':'Unnamed: 43']
df7.columns = df7.loc[0]
df7 = df7.drop(0)
df7.columns = ['cordialidade_do_motorista','entrega_no_prazo','ciclo_pedido_adequado','condicoes_de_frota','procedimentos_seguranca_na_entrega']
df7.index = df7.index.rename('ID')

df7 = pd.merge(df7,df_geral, on='ID', left_index=True)

df7.to_json(f'avaliacao_servico_entrega.json',orient="records")


df7.head()

,cordialidade_do_motorista,entrega_no_prazo,ciclo_pedido_adequado,condicoes_de_frota,procedimentos_seguranca_na_entrega,date_created
ID,,,,,,
1,1000,1000,1000,1000,1000,12/17/2020 01:46:57 PM
2,1000,1000,1000,1000,1000,12/17/2020 02:58:55 PM
3,1000,1000,1000,1000,1000,12/17/2020 02:22:27 PM
4,1000,1000,1000,1000,1000,12/17/2020 02:33:53 PM
5,1000,1000,1000,1000,1000,12/17/2020 01:52:50 PM


**Tratando e gerando tabela Avaliação em nota do serviço de entrega**

In [39]:
df8 = df.loc[:,'Em linhas gerais como considera o serviço de entrega da ATEM?':'Unnamed: 49']
df8.columns = df8.loc[0]
df8 = df8.drop(0)
#df8.columns = ['cinco ', 'quatro', 'tres', 'dois', 'um','comentario']
df8.index = df8.index.rename('ID')
df8['cinco'] = df8[5.0].astype(int)
df8['quatro'] = df8[4.0].astype(int)
df8['tres'] = df8[3.0].astype(int)
df8['dois'] = df8[2.0].astype(int)
df8['um'] = df8[1.0].astype(int)

aux=1000
df8['Nota'] =['5' if a != aux
              else '4' if b != aux
              else '3' if c != aux
              else '2' if d != aux
              else '1' if e != aux
              else 'Não Informado'
              for a,b,c,d,e in zip(df8['cinco'].to_list(), df8['quatro'].to_list(), df8['tres'].to_list(), df8['dois'].to_list(), df8['um'].to_list())]
               
df8 = df8.drop(columns=[5.0,4.0,3.0,2.0,1.0,'cinco','quatro','tres','dois','um'])
df8.columns = [ 'comentario','nota']

df8 = pd.merge(df8,df_geral, on='ID', left_index=True)

df8.to_json(f'avaliacao_nota_servico_entrega.json',orient="records")

df8.head()

,comentario,nota,date_created
ID,,,
1,1000,Não Informado,12/17/2020 01:46:57 PM
2,1000,Não Informado,12/17/2020 02:58:55 PM
3,1000,Não Informado,12/17/2020 02:22:27 PM
4,1000,Não Informado,12/17/2020 02:33:53 PM
5,1000,Não Informado,12/17/2020 01:52:50 PM


**Tabela Avaliação da retirada do produto na base da ATEM**

In [40]:
df9 = df.loc[:,'SE RETIRA responder as perguntas 8 e 9Em caso de retirada do produto na base da ATEM:':'Unnamed: 54']
df9.columns = df9.loc[0]
df9 = df9.drop(0)
df9.columns = ['cordialidade_do_atendimento','mercadoria_disponivel_no_prazo','tempo_do_ciclo_do_pedido_adequado','procedimentos_de_seguranca_na_entrega','comunicacao_de_agenda_eficiente']
df9.index = df9.index.rename('ID')

df9 = pd.merge(df9,df_geral, on='ID', left_index=True)

df9.to_json(f'avaliacao_retirada_na_base_atem.json',orient="records")

df9.head()

,cordialidade_do_atendimento,mercadoria_disponivel_no_prazo,tempo_do_ciclo_do_pedido_adequado,procedimentos_de_seguranca_na_entrega,comunicacao_de_agenda_eficiente,date_created
ID,,,,,,
1,5,4,5,5,5,12/17/2020 01:46:57 PM
2,1000,1000,1000,1000,1000,12/17/2020 02:58:55 PM
3,1000,1000,1000,1000,1000,12/17/2020 02:22:27 PM
4,1000,1000,1000,1000,1000,12/17/2020 02:33:53 PM
5,5,5,5,5,5,12/17/2020 01:52:50 PM


**Tratando e Gerando tabela Avaliação em nota da retirada do produto na base da ATEM**

In [42]:

df10 = df.loc[:,'Em linhas gerais como considera o serviço de retirada do produto na base da ATEM?':'Unnamed: 60']
df10.columns = df10.loc[0]
df10 = df10.drop(0)

df10.index = df10.index.rename('ID')

df10['cinco'] = df10[5.0].astype(int)
df10['quatro'] = df10[4.0].astype(int)
df10['tres'] = df10[3.0].astype(int)
df10['dois'] = df10[2.0].astype(int)
df10['um'] = df10[1.0].astype(int)

aux=1000
df10['Nota'] =['5' if a != aux
              else '4' if b != aux
              else '3' if c != aux
              else '2' if d != aux
              else '1' if e != aux
              else 'Não Informado'
              for a,b,c,d,e in zip(df10['cinco'].to_list(), df10['quatro'].to_list(), df10['tres'].to_list(), df10['dois'].to_list(), df10['um'].to_list())]
               
df10 = df10.drop(columns=[5.0,4.0,3.0,2.0,1.0,'cinco','quatro','tres','dois','um'])
df10.columns = ['comentario','nota']
df10 = pd.merge(df10,df_geral, on='ID', left_index=True)


df10.to_json(f'avaliacao_nota_retirada_na_base_atem.json',orient="records")

df10.head()

,comentario,nota,date_created
ID,,,
1,AS VEZES ATRASA FORA ISSO OTIMO ATENDIMENTO,4,12/17/2020 01:46:57 PM
2,1000,Não Informado,12/17/2020 02:58:55 PM
3,1000,Não Informado,12/17/2020 02:22:27 PM
4,1000,Não Informado,12/17/2020 02:33:53 PM
5,1000,5,12/17/2020 01:52:50 PM


**Tabela Avaliação em nota central de atendimento**

In [48]:
df11 = df.loc[:,'Quanto a Central de Atendimento da ATEM, classifique os seguintes itens:':'Unnamed: 63']
df11.columns = df11.loc[0]
df11 = df11.drop(0)
df11.columns = ['cordialidade','eficacia','eficiencia']
df11.index = df11.index.rename('ID')

df11 = pd.merge(df11,df_geral, on='ID', left_index=True)

df11.to_json(f'avaliacao_nota_atendimento_da_central.json',orient="records")

df11.head()

,cordialidade,eficacia,eficiencia,date_created
ID,,,,
1,1000,1000,1000,12/17/2020 01:46:57 PM
2,1000,1000,1000,12/17/2020 02:58:55 PM
3,1000,1000,1000,12/17/2020 02:22:27 PM
4,1000,1000,1000,12/17/2020 02:33:53 PM
5,5,5,5,12/17/2020 01:52:50 PM


**Tabela Avaliação central de atendimento**
* Aplicar tratamento no campo comentário sobre a nota - separar nota do comentario

In [49]:
df12 = df.loc[:,'Você já fez alguma reclamação ou solicitação na CA?Se SIM, responder P12 e P13. Se NÃO pular para P14':'Unnamed: 69']
df12.columns = df12.loc[0]
df12 = df12.drop(0)
df12.columns = ['fez_alguma_reclamacao_ou_solicitacao','quantas_vezes_abriu_chamado','recebeu_retorno_em_48hs','apresentaram_solucao_da_solicitacao','como_considera_servico_central','comentario_sobre_a_nota']
df12.index = df9.index.rename('ID')

df12 = pd.merge(df12,df_geral, on='ID', left_index=True)

#df11.to_json(f'avaliacao_central_atendimento.json',orient="records")


df12.head()

,fez_alguma_reclamacao_ou_solicitacao,quantas_vezes_abriu_chamado,recebeu_retorno_em_48hs,apresentaram_solucao_da_solicitacao,como_considera_servico_central,comentario_sobre_a_nota,date_created
ID,,,,,,,
1,NÃO,1000,1000,1000,1000,1000,12/17/2020 01:46:57 PM
2,1000,1000,1000,1000,1000,1000,12/17/2020 02:58:55 PM
3,1000,1000,1000,1000,1000,1000,12/17/2020 02:22:27 PM
4,1000,1000,1000,1000,1000,1000,12/17/2020 02:33:53 PM
5,SIM,Entre 2 e 5 vezes,SIM,SIM,Por que atribuiu esta nota? Comente,5,12/17/2020 01:52:50 PM


**Tabela de Avaliação das ações promocionais de marketing**

In [ ]:
df14 = df.loc[:,'Observações da Pesquisa: (Atendente)']
df14.columns = df14.loc[0]
df14 = df14.drop(0)

df14.index = df4.index.rename('ID')

df4 = pd.merge(df4,df_geral, on='ID', left_index=True)


df9.head()

0                        Open-Ended Response
1                                  MIKAELLEN
2    Cliente não tem interesse em responder.
3    Cliente não tem interesse em responder.
4    Cliente não tem interesse em responder.
Name: Observações da Pesquisa: (Atendente), dtype: object

In [51]:
df13 = df.loc[:,'(Somente posto ATEM e pontão ATEM) Quanto as ações promocionais de Marketing, você observa que:':'Unnamed: 72']
df13.columns = df13.loc[0]
df13 = df13.drop(0)
df13.columns = ['existe_interesses_dos_clientes_nas_acoes','existe_aumento_nas_vendas_no_periodo','existe_novos_clientes']

df13.index = df13.index.rename('ID')

df13 = pd.merge(df13,df_geral, on='ID', left_index=True)

df13.to_json(f'avaliacao_acoes_promocionais.json',orient="records")

df13.head()

,existe_interesses_dos_clientes_nas_acoes,existe_aumento_nas_vendas_no_periodo,existe_novos_clientes,date_created
ID,,,,
1,1000,1000,1000,12/17/2020 01:46:57 PM
2,1000,1000,1000,12/17/2020 02:58:55 PM
3,1000,1000,1000,12/17/2020 02:22:27 PM
4,1000,1000,1000,12/17/2020 02:33:53 PM
5,1000,1000,1000,12/17/2020 01:52:50 PM


#Envio do dados

In [ ]:
for item in data_list:
  send_elasticsearch('biblioteca_anuncios',
  item['id'],
  item)